In [6]:
import pandas as pd
from sqlalchemy import create_engine, text
import sys
import os
from db_conector import get_db_engine, get_db_session, get_db_connection

sys.path.append(os.path.join(os.getcwd(), 'orm'))

from db_conector import get_db_engine, get_db_session
from modelos import *

print("Conexion Configurada!!")


Conexion Configurada!!


In [7]:
import pandas as pd
from sqlalchemy import create_engine, text
import sys
import os

sys.path.append(os.path.join(os.getcwd(), 'orm'))

from db_conector import get_db_engine, get_db_session
from modelos import *

print("Conexion Configurada!!")


Conexion Configurada!!


In [8]:
# Probar la conexión a la base de datos
try:
    engine = get_db_engine()
    with get_db_connection() as conn:
        # Probar una consulta simple
        result = conn.execute(text("SELECT 1 as test"))
        test_result = result.fetchone()
        print(f"Conexión exitosa: {test_result}")
        
        # Obtener lista de tablas
        result = conn.execute(text("""
            SELECT table_name 
            FROM information_schema.tables 
            WHERE table_schema = 'public'
            ORDER BY table_name
        """))
        tablas = result.fetchall()
        print(f"\nTablas disponibles en la base de datos:")
        for tabla in tablas:
            print(f"- {tabla[0]}")
            
except Exception as e:
    print(f"Error de conexión: {e}")

Conexión exitosa: (1,)

Tablas disponibles en la base de datos:
- carrito
- categorias
- detalle_ordenes
- dim_clientes
- dim_fecha
- dim_productos
- dim_ubicaciones
- direcciones_envio
- fact_ventas
- historial_pagos
- metodos_pago
- ordenes
- ordenes_metodos_pago
- productos
- resenas_productos
- usuarios


In [18]:
# Cargar todas las tablas en dataframes
def cargar_tablas_en_dataframes():
    """
    Carga todas las tablas de la base de datos en dataframes de pandas
    """
    dataframes = {}
    
    # Lista de tablas a cargar
    tablas = [
        'usuarios',
        'categorias', 
        'productos',
        'ordenes',
        'detalle_ordenes',
        'direcciones_envio',
        'carrito',
        'metodos_pago',
        'ordenes_metodos_pago',
        'resenas_productos',
        'historial_pagos'
    ]
    
    try:
        with get_db_connection() as conn:
            for tabla in tablas:
                try:
                    # Cargar cada tabla en un dataframe
                    query = f"SELECT * FROM {tabla}"
                    df = pd.read_sql(query, conn)
                    dataframes[tabla] = df
                    print(f"Tabla '{tabla}' cargada: {len(df)} filas, {len(df.columns)} columnas")
                except Exception as e:
                    print(f"Error cargando tabla '{tabla}': {e}")
                    
        return dataframes
        
    except Exception as e:
        print(f"Error general al cargar tablas: {e}")
        return {}

# Ejecutar la carga de tablas
print("Cargando tablas en dataframes...")
dfs = cargar_tablas_en_dataframes()

Cargando tablas en dataframes...
Tabla 'usuarios' cargada: 1000 filas, 7 columnas
Tabla 'categorias' cargada: 12 filas, 3 columnas
Tabla 'productos' cargada: 36 filas, 6 columnas
Tabla 'ordenes' cargada: 10000 filas, 5 columnas
Tabla 'detalle_ordenes' cargada: 10000 filas, 5 columnas
Tabla 'direcciones_envio' cargada: 1000 filas, 10 columnas
Tabla 'carrito' cargada: 5000 filas, 5 columnas
Tabla 'metodos_pago' cargada: 7 filas, 3 columnas
Tabla 'ordenes_metodos_pago' cargada: 10000 filas, 4 columnas
Tabla 'direcciones_envio' cargada: 1000 filas, 10 columnas
Tabla 'carrito' cargada: 5000 filas, 5 columnas
Tabla 'metodos_pago' cargada: 7 filas, 3 columnas
Tabla 'ordenes_metodos_pago' cargada: 10000 filas, 4 columnas
Tabla 'resenas_productos' cargada: 10000 filas, 6 columnas
Tabla 'historial_pagos' cargada: 10000 filas, 6 columnas
Tabla 'resenas_productos' cargada: 10000 filas, 6 columnas
Tabla 'historial_pagos' cargada: 10000 filas, 6 columnas


In [20]:
# Explorarcion de los dataframes cargados
def explorar_dataframes(dataframes_dict):

    print("="*60)
    print("RESUMEN DE DATAFRAMES CARGADOS")
    print("="*60)
    
    for nombre_tabla, df in dataframes_dict.items():
        print(f"\n  TABLA: {nombre_tabla.upper()}")
        print(f"   Dimensiones: {df.shape[0]} filas x {df.shape[1]} columnas")
        print(f"   Columnas: {list(df.columns)}")
        
        # Mostrar primeras 3 filas si hay datos
        if len(df) > 0:
            print(f"        Primeras 3 filas:")
            print(df.head(3).to_string(index=False))
        else:
            print(" Tabla vacía")
        print("-" * 50)

# Ejecutar exploración
if dfs:
    explorar_dataframes(dfs)
else:
    print("No se pudieron cargar los dataframes")

RESUMEN DE DATAFRAMES CARGADOS

  TABLA: USUARIOS
   Dimensiones: 1000 filas x 7 columnas
   Columnas: ['id', 'nombre', 'apellido', 'dni', 'email', 'contrasena', 'fecha_registro']
        Primeras 3 filas:
 id       nombre  apellido      dni                            email    contrasena             fecha_registro
  1      Encarna   Donaire 49877134      encarna.donaire1@correo.com contrasena123 2025-07-28 19:47:54.263673
  2 Jose Ignacio   Canales 98778810 jose ignacio.canales2@correo.com contrasena123 2025-07-28 19:47:54.263673
  3         José Sebastián 80095714       josé.sebastián3@correo.com contrasena123 2025-07-28 19:47:54.263673
--------------------------------------------------

  TABLA: CATEGORIAS
   Dimensiones: 12 filas x 3 columnas
   Columnas: ['id', 'nombre', 'descripcion']
        Primeras 3 filas:
 id         nombre                                                      descripcion
  1    Electrónica  Teléfonos, computadoras, accesorios y dispositivos inteligentes
  2  

In [ ]:
#  Cantidad de nulos por tabla
def nulos_rapido(dataframes_dict):

    for nombre, df in dataframes_dict.items():
        nulos = df.isnull().sum().sum()
        print(f"{nombre}: {nulos} valores nulos")

# Usar así:
nulos_rapido(dfs)

usuarios: 0 valores nulos
categorias: 0 valores nulos
productos: 0 valores nulos
ordenes: 0 valores nulos
detalle_ordenes: 0 valores nulos
direcciones_envio: 0 valores nulos
carrito: 0 valores nulos
metodos_pago: 0 valores nulos
ordenes_metodos_pago: 0 valores nulos
resenas_productos: 0 valores nulos
historial_pagos: 0 valores nulos
